<a href="https://colab.research.google.com/github/simonsny/image_background_removal/blob/main/bg_removal_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import cv2 as cv
import numpy as np
import torch
import torch.nn
from torchvision import transforms
from tqdm import tqdm

Get the Binary Mask with U2Net

In [ ]:
# clone the repository
%cd /content/drive/MyDrive/BeCode/Faktion/
if not os.path.exists('U-2-Net'):
  !git clone https://github.com/NathanUA/U-2-Net.git
%cd U-2-Net
#!python u2net_test.py

[Errno 2] No such file or directory: '/content/drive/MyDrive/BeCode/Faktion/'
/content
Cloning into 'U-2-Net'...
remote: Enumerating objects: 800, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 800 (delta 114), reused 158 (delta 95), pack-reused 604
Receiving objects: 100% (800/800), 30.71 MiB | 16.95 MiB/s, done.
Resolving deltas: 100% (360/360), done.
/content/U-2-Net


Getting the inference from U2Net to get the mask.

In [ ]:
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET # full size version 173.6 MB
from model import U2NETP # small version u2net 4.7 MB

# normalize the predicted SOD probability map
def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

def save_output(image_name,pred,d_dir):

    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    img_name = image_name.split(os.sep)[-1]
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

    pb_np = np.array(imo)

    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]

    imo.save(d_dir+imidx+'.png')

def main():

    # --------- 1. get image path and name ---------
    model_name='u2net'#u2netp



    image_dir = os.path.join(os.getcwd(), 'test_data', 'test_images')
    prediction_dir = os.path.join(os.getcwd(), 'test_data', model_name + '_results' + os.sep)
    model_dir = os.path.join(os.getcwd(), 'saved_models', model_name, model_name + '.pth')

    img_name_list = glob.glob(image_dir + os.sep + '*')
    print(img_name_list)

    # --------- 2. dataloader ---------
    #1. dataloader
    test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                        lbl_name_list = [],
                                        transform=transforms.Compose([RescaleT(320),
                                                                      ToTensorLab(flag=0)])
                                        )
    test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                        batch_size=1,
                                        shuffle=False,
                                        num_workers=1)

    # --------- 3. model define ---------
    if(model_name=='u2net'):
        print("...load U2NET---173.6 MB")
        net = U2NET(3,1)
    elif(model_name=='u2netp'):
        print("...load U2NEP---4.7 MB")
        net = U2NETP(3,1)

    if torch.cuda.is_available():
        net.load_state_dict(torch.load(model_dir))
        net.cuda()
    else:
        net.load_state_dict(torch.load(model_dir, map_location='cpu'))
    net.eval()

    # --------- 4. inference for each image ---------
    for i_test, data_test in enumerate(test_salobj_dataloader):

        print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

        inputs_test = data_test['image']
        inputs_test = inputs_test.type(torch.FloatTensor)

        if torch.cuda.is_available():
            inputs_test = Variable(inputs_test.cuda())
        else:
            inputs_test = Variable(inputs_test)

        d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

        # normalization
        pred = d1[:,0,:,:]
        pred = normPRED(pred)

        # save results to test_results folder
        if not os.path.exists(prediction_dir):
            os.makedirs(prediction_dir, exist_ok=True)
        save_output(img_name_list[i_test],pred,prediction_dir)

        del d1,d2,d3,d4,d5,d6,d7

if __name__ == "__main__":
    main()

Generate the Trimap from the Binary Mask

In [ ]:
def generate_trimap(mask_path, open_size=10, mask_margin=10):
    mask = cv2.imread(mask_path, 0)

    # make sure if the alpha_matte array has 2 dimensions 
    assert mask.ndim == 2

    # compute each region
    ## get foreground region
    foreground = ((255 - mask_margin) < mask)

    ## get background region
    background = (mask < mask_margin)

    ## get unknown region: outside of background and foreground
    unknown = ~(foreground + background)

    ## dilate unknown area with elliptical kernel.
    unknown = cv2.dilate(
        unknown.astype(np.uint8),
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_size, open_size))
    ).astype(np.bool)

    # draw trimap
    trimap = np.zeros_like(mask)
    trimap[foreground] = 255
    trimap[unknown] = 127

    # save trimap
    #cv2.imwrite(dst_path, trimap)
    cv2.imwrite(f"/content/drive/MyDrive/BeCode/Faktion/Flask/static/trimap/{mask_path.split('/')[-1]}",trimap)
    return mask_path.split('/')[-1]

Inference with DIM pretrained model

In [ ]:
from torchvision import transforms
%cd /content/drive/MyDrive/Faktion/model_code
from config import device
from utils import ensure_folder


IMG_FOLDER = '/content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Image'
TRIMAP_FOLDERS = ['/content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap', '/content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon']
OUTPUT_FOLDERS = ['/content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Output',
                  '/content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon-Output']


print(1)

checkpoint = '/content/drive/MyDrive/Faktion/BEST_checkpoint.tar'
checkpoint = torch.load(checkpoint)
print(2)
model = checkpoint['model'].module
model = model.to(device)
model.eval()
print('model eval')


files = [f for f in os.listdir(IMG_FOLDER)]

for file in tqdm(files):
  print('for loop 1')
  filename = os.path.join(IMG_FOLDER, file)
  img = cv.imread(filename)
  print(img.shape)
  h, w = img.shape[:2]

  x = torch.zeros((1, 4, h, w), dtype=torch.float)
  image = img[..., ::-1]  # RGB
  image = transforms.ToPILImage()(image)

  transformer = transforms.Compose(
      [
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  image = transformer(image)
  x[0:, 0:3, :, :] = image

  for i in range(2):
    print('for loop 2')
    filename = os.path.join(TRIMAP_FOLDERS[i], file.replace('.jpg', '.png'))
    print('reading {}...'.format(filename))
    trimap = cv.imread(filename, 0)
    print(trimap)
    x[0:, 3, :, :] = torch.from_numpy(trimap.copy() / 255.)
    # print(torch.max(x[0:, 3, :, :]))
    # print(torch.min(x[0:, 3, :, :]))
    # print(torch.median(x[0:, 3, :, :]))

    # Move to GPU, if available
    x = x.type(torch.FloatTensor).to(device)

    with torch.no_grad():
        pred = model(x)

    pred = pred.cpu().numpy()
    pred = pred.reshape((h, w))

    pred[trimap == 0] = 0.0
    pred[trimap == 255] = 1.0

    out = (pred.copy() * 255).astype(np.uint8)

    filename = os.path.join(OUTPUT_FOLDERS[i], file)
    cv.imread(filename)
    cv.imwrite(filename, out)
    print('wrote {}.'.format(filename))

/content/drive/.shortcut-targets-by-id/1pbfU2Nkn_OKUrAgN_cnPqhS91KoSQBr8/Faktion/model_code
1


/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attri

2
model eval
for loop 1
(300, 400, 3)
for loop 2
reading /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap/n07930864_7479.png...
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
wrote /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Output/n07930864_7479.jpg.
for loop 2
reading /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon/n07930864_7479.png...


  0%|          | 1/10553 [00:00<2:55:31,  1.00it/s]

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
wrote /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon-Output/n07930864_7479.jpg.
for loop 1
(285, 400, 3)
for loop 2
reading /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap/n03710721_7570.png...
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
wrote /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Output/n03710721_7570.jpg.
for loop 2
reading /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon/n03710721_7570.png...


...

  0%|          | 17/10553 [00:21<3:39:49,  1.25s/it]

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
wrote /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon-Output/n07718747_470.jpg.
for loop 1
(400, 300, 3)
for loop 2
reading /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap/ILSVRC2012_test_00015074.png...
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
wrote /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Output/ILSVRC2012_test_00015074.jpg.
for loop 2
reading /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon/ILSVRC2012_test_00015074.png...
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


  0%|          | 18/10553 [00:22<3:42:27,  1.27s/it]

wrote /content/drive/MyDrive/Faktion/Datasets/DUTS-TR/DUTS-TR-Trimap-Simon-Output/ILSVRC2012_test_00015074.jpg.
for loop 1


KeyboardInterrupt: ignored